## 딥러닝 활용 (4)
### : 생성적 적대 신경망(GAN: Generative Adversarial Network)

In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import numpy as np
import matplotlib.pyplot as plt

In [7]:
#생성자
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))  #128과 100은 랜덤
generator.add(BatchNormalization())
generator.add(Reshape((7,7,128)))
generator.add(UpSampling2D())
generator.add(Conv2D(64,kernel_size=5, padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1,kernel_size=5, padding='same',activation='tanh'))

In [9]:
#판별자- 진짜 or 가짜 구분만하기, 학습X
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding = 'same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1,activation='sigmoid'))

discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable=False

In [14]:
#G와 D이어주는 모델 만들기
ginput = Input(shape=(100,)) #생성자에 입력할 랜덤한 100개의 벡터를 만든다
dis_output = discriminator(generator(ginput)) #벡터를 G에 넣은 값을 판별한 값을 넣는다
gan = Model(ginput, dis_output)
gan.compile(loss='binary_crossentropy',optimizer='adam')

#실행 함수 선언
def gan_train(epoch, batch_size, saving_interval):
    #테스트 과정 필요없이 이미지만 사용할 것이므로 X_train만 호출
    (X_train,_),(_,_) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 28, 28,1).astype('float32')
    #x의 값을 -1 ~ 1 사이의 값으로 바꿈
    X_train = (X_train - 127.5) / 127.5
    true = np.ones((batch_size, 1))
    fake = np.zeros((batch_size,1))
    
    for i in range(epoch):
        #실제 데이터를 판별자에 입력
        idx = np.random.randint(0,X_train.shape[0], batch_size)
        imgs = X_train[idx]
        d_loss_real = discriminator.train_on_batch(imgs, true)
        
        #가상 이미지를 판별자에 입력
        noise = np.random.normal(0,1,(batch_size,100))
        gen_imgs = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        
        #판별자와 생성자의 오차 계산
        d_loss = 0.5*np.add(d_loss_real, d_loss_fake)
        g_loss = gan.train_on_batch(noise, true)
        
        print('epoch:%d'%i, "d_loss:%.4f"%d_loss, 'g_loss: %.4f'%g_loss)

In [ ]:
gan_train(4001,32,200)

epoch:0 d_loss:0.5147 g_loss: 0.6594
epoch:1 d_loss:0.3356 g_loss: 0.2508
epoch:2 d_loss:0.3492 g_loss: 0.0418
epoch:3 d_loss:0.9706 g_loss: 0.0114
epoch:4 d_loss:1.3944 g_loss: 0.0314
epoch:5 d_loss:1.1769 g_loss: 0.2054
epoch:6 d_loss:0.5707 g_loss: 0.9938
epoch:7 d_loss:0.6644 g_loss: 1.3914
epoch:8 d_loss:0.7172 g_loss: 1.2750
epoch:9 d_loss:0.6283 g_loss: 1.0159
epoch:10 d_loss:0.5194 g_loss: 0.8054
epoch:11 d_loss:0.4298 g_loss: 0.6941
epoch:12 d_loss:0.4192 g_loss: 0.5726
epoch:13 d_loss:0.3829 g_loss: 0.5650
epoch:14 d_loss:0.3404 g_loss: 0.4817
epoch:15 d_loss:0.2714 g_loss: 0.5616
epoch:16 d_loss:0.2208 g_loss: 0.5331
epoch:17 d_loss:0.2872 g_loss: 0.6567
epoch:18 d_loss:0.1978 g_loss: 0.4466
epoch:19 d_loss:0.2934 g_loss: 0.4634
epoch:20 d_loss:0.3167 g_loss: 0.2029
epoch:21 d_loss:0.5719 g_loss: 0.1507
epoch:22 d_loss:0.4473 g_loss: 0.1813
epoch:23 d_loss:0.5263 g_loss: 0.2867
epoch:24 d_loss:0.6638 g_loss: 0.3634
epoch:25 d_loss:1.1769 g_loss: 0.2979
epoch:26 d_loss:1.1095

epoch:213 d_loss:0.5643 g_loss: 1.6369
epoch:214 d_loss:0.5930 g_loss: 1.5507
epoch:215 d_loss:0.5158 g_loss: 1.9888
epoch:216 d_loss:0.5671 g_loss: 1.9476
epoch:217 d_loss:0.5535 g_loss: 1.8731
epoch:218 d_loss:0.5474 g_loss: 1.5582
epoch:219 d_loss:0.4898 g_loss: 1.5571
epoch:220 d_loss:0.5082 g_loss: 1.4718
epoch:221 d_loss:0.7159 g_loss: 1.4179
epoch:222 d_loss:0.5589 g_loss: 1.8107
epoch:223 d_loss:0.5515 g_loss: 1.8433
epoch:224 d_loss:0.5372 g_loss: 1.4822
epoch:225 d_loss:0.6599 g_loss: 1.6873
epoch:226 d_loss:0.6251 g_loss: 1.6627
epoch:227 d_loss:0.4714 g_loss: 1.2285
epoch:228 d_loss:0.5480 g_loss: 1.5803
epoch:229 d_loss:0.5028 g_loss: 1.3899
epoch:230 d_loss:0.6025 g_loss: 1.6159
epoch:231 d_loss:0.4933 g_loss: 1.8274
epoch:232 d_loss:0.4268 g_loss: 1.6899
epoch:233 d_loss:0.4435 g_loss: 1.8679
epoch:234 d_loss:0.3874 g_loss: 1.9025
epoch:235 d_loss:0.4053 g_loss: 1.6885
epoch:236 d_loss:0.5217 g_loss: 1.6078
epoch:237 d_loss:0.3857 g_loss: 1.5564
epoch:238 d_loss:0.4128 g

epoch:424 d_loss:0.4537 g_loss: 1.8759
epoch:425 d_loss:0.4801 g_loss: 2.1280
epoch:426 d_loss:0.5643 g_loss: 1.4673
epoch:427 d_loss:0.4218 g_loss: 2.1910
epoch:428 d_loss:0.3292 g_loss: 2.5611
epoch:429 d_loss:0.4005 g_loss: 2.1432
epoch:430 d_loss:0.4546 g_loss: 2.1966
epoch:431 d_loss:0.5818 g_loss: 1.9399
epoch:432 d_loss:0.4167 g_loss: 2.4726
epoch:433 d_loss:0.5405 g_loss: 1.5855
epoch:434 d_loss:0.6025 g_loss: 1.6853
epoch:435 d_loss:0.4261 g_loss: 2.4373
epoch:436 d_loss:0.4488 g_loss: 2.1347
epoch:437 d_loss:0.4092 g_loss: 2.0689
epoch:438 d_loss:0.5481 g_loss: 2.1817
epoch:439 d_loss:0.4893 g_loss: 2.4962
epoch:440 d_loss:0.6649 g_loss: 1.5497
epoch:441 d_loss:0.5363 g_loss: 1.3209
epoch:442 d_loss:0.6370 g_loss: 1.3631
epoch:443 d_loss:0.3800 g_loss: 2.2708
epoch:444 d_loss:0.3948 g_loss: 2.5049
epoch:445 d_loss:0.6463 g_loss: 2.5451
epoch:446 d_loss:0.5813 g_loss: 2.0705
epoch:447 d_loss:0.4360 g_loss: 1.4619
epoch:448 d_loss:0.4240 g_loss: 1.2360
epoch:449 d_loss:0.5653 g